# SCORE OBJECTS

An overview of musx objects that facilitate musical score generation.
<hr style="height:1px;color:gray">

Notebook setup:

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from musx import Event, Note, MidiEvent, Seq, Score, version, between, Pitch, MidiFile, setmidiplayer, playfile, rescale
from musx.mxml import notation
print(f"musx.version: {version}")

This notebook generates MIDI files and automatically plays them using [fluidsynth](https://www.fluidsynth.org/download/) and the [MuseScore_General.sf3](https://ftp.osuosl.org/pub/musescore/soundfont/MuseScore_General) sound font. See [INSTALL.md](https://github.com/musx-admin/musx/blob/main/INSTALL.md) for how to install a terminal-based MIDI player to use with musx.  If you don't have a player installed you can access the output files in the same directory as this notebook:

In [ ]:
setmidiplayer("fluidsynth -iq -g1 /Users/taube/Music/SoundFonts/MuseScore_General.sf2")
print('OK!')

## Sound Events

Musx defines several classes that bundle sound parameters and facilitate the creation and playback of musical sounds:

### Event

`Event` is a base class that enables any subclass to be added to `Seq` containers. It provides a single sound parameter, `time`, that stores start time (in whatever units) of the sounding event:

In [ ]:
ev = Event(1.2)
print(ev)
print(ev.time)

Defining new sound objects is straight-forward. As an example, here is the implementation of an simple OSC message that could be used to send OSC data to SuperCollider or other external apps (see the OSC tutorial in the same directory as this notebook):

In [ ]:
class OSC(Event):
    def __init__(self, address, time, *data):
        super().__init__(time)
        self.addr = address
        self.data = [*data]
    def __str__(self):
        return f"<OSC: '{self.addr}' {self.time} {self.data} {hex(id(self))}>"
    def __repr__(self):
        return f"OSC('{self.addr}', {self.time}, {', '.join([repr(d) for d in self.data])})"

osc = OSC('musx/synth', 2001, [220, 880], .5, "x")

print(f"{str(osc)}\n\n{repr(osc)}")
print(f"\ntime: {osc.time}, address: '{osc.addr}'', data: {osc.data}")

### Note

A `Note` is a flexible sound event that can be used in different contexts. For example, if a Note is passed to methods in the MIDI or Csound backends the note data is automatically converted to the format supported by that module. If a MusicXML file is loaded its symbolic note information will be represented in a Note. 

`Note(time=0.0, duration=1.0, pitch=60, amplitude=0.5, instrument=0 ...)`

* The `time` of a note is its start time in a score, typically (but not necessarily) in seconds.
* The `duration` is the length of time that the sound lasts, typically (but not necessarily) in seconds.
* The `pitch` is the frequency of the sound. This parameter accept Pitch objects, integer key numbers, floating point key numbers (microtuning), chords (note lists) and rests (empty Pitches):
* The `amplitude` is the loudness of the sound and ranges from 0.0 to 1.0.
* The `instrument` is a timbre designation of some kind, defaults to 0. If you are generating midi then this value would be a channel integer 0 to 15, inclusive.

The values assigned to Note parameters are flexible and ultimately depend on the 'back end' that is being composed for:

In [ ]:
n=Note(time=1, duration=3, pitch=Pitch("C#3"), amplitude=.5)
print(n)
print(n.time, n.duration, n.pitch, n.amplitude)

If a list of pitches or key numbers is given, the Note will be tagged as a chord and contain the first Pitch plus the remaining pitches converted into child Notes, each child containing the same attribute values as the parent except for pitch. A pitch can be detected by its `tag` attribute, and the text display will show multiple pitches delimited by ':'. If the note is tagged as a chord the `chord()` function will return the complete list of note objects:

In [ ]:
n=Note(pitch=[60, 64, 67])
print(n)
print(n.tag)
print(n.pitch)
print(n.chord())

If the pitch parameter receives an empty Pitch the Note will be tagged as a rest (R):

In [ ]:
n=Note(pitch=Pitch())
print(n)
print(n.tag)
print(n.pitch)

Notes that are created by loading a MusicXML file will include a dictionary of MusicXML markup. In this example the Note markup is a voice assignment:

In [ ]:
hello = notation.load("support/HelloWorld.musicxml")
hello.print()

In [ ]:
n=Note(pitch=Pitch("Bb3"))
n.set_mxml("voice", 1)
print(n)

### MidiEvent

A `MidiEvent` associates low-level midi messages (raw byte lists) with the inherited time attribute so they can be added to sequences, sorted, etc. The MidiEvent class contains factory methods to wrap any midi message, including meta messages.
Avoid working with explicit `MidiEvent.note_on()` and `MidiEvent.note_off()` messages since the Note object does this conversion for you automatically when it is written to a midi file:

In [ ]:
# musx MIDI numbers start at 0, not 1, your midi keyboard would show 'channel 1' and 'program 1' (piano):

pc = MidiEvent.program_change(chan=0, prog=0,time=10.0)
print(str(pc))
print(repr(pc))
print(pc.time)
print(pc.message)

## Event containers

### Seq

The `Seq` object is a container that maintains a time sorted sequence of Event objects, e.g. instances of Note, MidiEvent, or any other subclass. When objects are added to the sequence they are automatically inserted at the proper timepoint. This cell generates random note times, and then adds the notes to a sequence to sort them according to their start times:

In [ ]:
ran = [round(between(0.0, 10), 2) for _ in range(8)]
print(f"random times: {ran}")
seq = Seq()
for r in ran:
    seq.add(Note(r))
seq.print()

Adding multiple events with the same time will appear in the order they were added to the sequence:

In [ ]:
t = seq[-1].time
for k in range(61,66):
    seq.add(Note(t, pitch=k))
seq.print()

A Seq is an Python iterable so its events can be mapped, sliced, etc.:

In [ ]:
print(len(seq))
print(seq[4:6])
for s in seq:
    print(s)

### Score

A `Score` is a container that acts as conductor: it manages a scheduling queue containing one or more *part composers* (python generators) that compute sound objects and add them to one (or more) Seq object maintained by the score. The part composer in this example adds notes whose pitches are ASCII values (0-127) of a given text string. A score is created and given an empty sequence to hold the generated composition. When Score.compose() method is called one instance (in this case) of the helloworld() part composer is passed in. The score then runs the composer, which creates one note for each letter in the text and adds it to the score.  When all the letters are rendered the composer stops and the score is complete:

In [ ]:
# ["Hello, World!", "Hallo Welt!", "Salut le Monde!", "¡Hola mundo!", "Halò, a Shaoghail!", 
# "Ciao Mondo!", "Zdravo Svete!", "Ahoj, svet!", "Pozdravljen svet!", "Hallå världen!"]

def helloworld(score, text, rate, amp):
    for char in text:
        key = rescale(ord(char) % 127, 0, 127, 35, 81)
        note = Note(time=score.now, duration=rate, pitch=key, amplitude=amp, instrument=9)
        score.add(note)
        yield rate

score = Score(out=Seq())
score.compose( helloworld(score, "Pozdravljen svet!", .5, .8) )
print(score.out)
score.out.print()

### MidiFile

`MidiFile(path, tracks=[], divs=480)`

A MidiFile is a 'backend' interface for writing to, or reading from, MIDI data stored in files on disk. To write a MIDI file, provide a filename and one or more MIDI sequence(s). If Note sequences are added to a midi file, the Note data will be automatically converted into MIDI messages:

In [ ]:
file = MidiFile("helloworld.mid", tracks=score.out).write()
print(f"Wrote '{file.pathname}'.")

Once the file has been written it can be played by any MIDI compatible software. If you have installed a terminal-based MIDI player (see top of this file) you can play the file without leaving Python:

In [ ]:
playfile(file.pathname)

#### Reading MIDI files

Use the `MidiFile.load()` function to load MIDI data into musx from a file on the disk. When you load a MidiFile each of its tracks will be converted into a musx sequence:

In [ ]:
here = %pwd
fullpathname = here + "/helloworld.mid"
infile = MidiFile(fullpathname).read()
print(infile.tracks[0])
infile.tracks[0].print()

## See Also

[partcomposers.ipynb](./partcomposers.ipynb)

Examples in the demos directory